https://medium.com/@sagieppel/train-fine-tune-segment-anything-2-sam-2-in-60-lines-of-code-928dd29a63b3

In [1]:
# segment image region using  fine tune model
# See Train.py on how to fine tune/train the model
import numpy as np
import torch
import cv2
import os
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

OLD_GPU False, USE_FLASH_ATTN True, MATH_KERNEL_ON False


In [2]:
# use bfloat16 for the entire script (memory efficient)
torch.autocast(device_type="cuda", dtype=torch.bfloat16).__enter__()

In [3]:
image_path = r"sample_image.jpg" # path to image
mask_path = r"sample_mask.png" # path to mask, the mask will define the image region to segment
num_samples = 30 # number of points/segment to sample

In [4]:
def read_image(image_path, mask_path): # read and resize image and mask
        img = cv2.imread(image_path)[...,::-1]  # read image
        mask = cv2.imread(mask_path,0)
        r = np.min([1024 / img.shape[1], 1024 / img.shape[0]])
        img = cv2.resize(img, (int(img.shape[1] * r), int(img.shape[0] * r)))
        mask = cv2.resize(mask, (int(mask.shape[1] * r), int(mask.shape[0] * r)),interpolation=cv2.INTER_NEAREST)
        return img, mask
def get_points(mask,num_points): # Sample points inside the input mask
        points=[]
        for i in range(num_points):
            coords = np.argwhere(mask > 0)
            yx = np.array(coords[np.random.randint(len(coords))])
            points.append([[yx[1], yx[0]]])
        return np.array(points)

In [5]:
# read image and sample points
image,mask = read_image(image_path, mask_path)
input_points = get_points(mask,num_samples)


In [ ]:
# Load model you need to have pretrained model already made
sam2_checkpoint = "sam2_hiera_small.pt" # "sam2_hiera_large.pt"
model_cfg = "sam2_hiera_s.yaml" # "sam2_hiera_l.yaml"
sam2_model = build_sam2(model_cfg, sam2_checkpoint, device="cuda")

In [6]:
# Load model you need to have pretrained model already made
sam2_checkpoint = "sam2_hiera_large.pt" # path to model weight
model_cfg = "sam2_hiera_l.yaml" # model config
sam2_model = build_sam2(model_cfg, sam2_checkpoint, device="cuda")

In [7]:
# Build net and load weights
predictor = SAM2ImagePredictor(sam2_model)
predictor.model.load_state_dict(torch.load("l_model_6047.torch"))


<All keys matched successfully>

In [8]:
# predict mask

with torch.no_grad():
    predictor.set_image(image)
    masks, scores, logits = predictor.predict(
        point_coords=input_points,
        point_labels=np.ones([input_points.shape[0],1])
    )


D:\workspace\20240810_Segment_Anything_2\segment-anything-2\sam2\modeling\backbones\hieradet.py:68: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  x = F.scaled_dot_product_attention(


In [12]:
# Short predicted masks from high to low score

np_masks = np.array(masks[:,0])
np_scores = scores[:,0]
shorted_masks = np_masks[np.argsort(np_scores)][::-1]


In [14]:
np_masks = np.array(masks[:,0].numpy()) if isinstance(masks, torch.Tensor) else np.array(masks[:,0])
np_scores = scores[:,0].float().numpy() if isinstance(scores, torch.Tensor) else np.array(scores[:,0])
shorted_masks = np_masks[np.argsort(np_scores)][::-1]

In [15]:
# Stitch predicted mask into one segmentation mask

seg_map = np.zeros_like(shorted_masks[0],dtype=np.uint8)
occupancy_mask = np.zeros_like(shorted_masks[0],dtype=bool)
for i in range(shorted_masks.shape[0]):
    mask = shorted_masks[i]
    if (mask*occupancy_mask).sum()/mask.sum()>0.15: continue
    mask[occupancy_mask]=0
    seg_map[mask]=i+1
    occupancy_mask[mask]=1

IndexError: arrays used as indices must be of integer (or boolean) type

In [16]:
seg_map = np.zeros_like(shorted_masks[0], dtype=np.uint8)
occupancy_mask = np.zeros_like(shorted_masks[0], dtype=bool)

for i in range(shorted_masks.shape[0]):
    mask = shorted_masks[i]
    if (mask * occupancy_mask).sum() / mask.sum() > 0.15:
        continue
    
    # Convert mask to boolean when needed
    mask_bool = mask.astype(bool)
    
    mask_bool[occupancy_mask] = False  # Set overlapping areas to False in the mask
    seg_map[mask_bool] = i + 1         # Use boolean mask to index seg_map
    occupancy_mask[mask_bool] = True   # Update occupancy_mask

In [17]:
# create colored annotation map
height, width = seg_map.shape

# Create an empty RGB image for the colored annotation
rgb_image = np.zeros((height, width, 3), dtype=np.uint8)
# Map each class number to a random  color


for id_class in range(1,seg_map.max()+1):
    rgb_image[seg_map == id_class] = [np.random.randint(255), np.random.randint(255), np.random.randint(255)]

# save and display

cv2.imwrite("annotation.png",rgb_image)
cv2.imwrite("mix.png",(rgb_image/2+image/2).astype(np.uint8))

cv2.imshow("annotation",rgb_image)
cv2.imshow("mix",(rgb_image/2+image/2).astype(np.uint8))
cv2.imshow("image",image)
cv2.waitKey()

-1